In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv

load_dotenv("../../.env.localhost")

# S3 init

In [ ]:
from functools import lru_cache
import os

import boto3

API_SITE_BUCKET = os.getenv("API_SITE_ID")


@lru_cache
def get_s3_client():
    _session = boto3.session.Session()
    return _session.client(
        "s3",
        region_name=os.getenv("DO_REGION_NAME"),
        endpoint_url=os.getenv("DO_ENDPOINT_URL"),
        aws_access_key_id=os.getenv("DO_SPACES_KEY"),
        aws_secret_access_key=os.getenv("DO_SPACES_SECRET"),
    )


def presigned_url_to_put(
    key, client=None, mime_type=None, bucket=API_SITE_BUCKET, expires_in=300
):
    return client.generate_presigned_url(
        ClientMethod="put_object",
        Params={"Bucket": bucket, "Key": key, "ContentType": mime_type},
        ExpiresIn=expires_in,
    )

s3client = get_s3_client()

response = s3client.list_buckets()
for space in response['Buckets']:
    print(space['Name'])
print(f"\nusing Bucket {API_SITE_BUCKET}\n")
response = s3client.list_objects(Bucket=API_SITE_BUCKET)
for obj in response['Contents']:
    print(obj['Key'])

# Site Client Session

In [ ]:
from nmdc_runtime.util import nmdc_jsonschema, nmdc_jsonschema_validate

In [ ]:
from nmdc_runtime.pipelines.core import run_config_frozen__preset_normal_env
from nmdc_runtime.resources.mongo import get_mongo


mongo = get_mongo(run_config_frozen__preset_normal_env)
db = mongo.db

In [ ]:
from nmdc_runtime.pipelines.core import run_config_frozen__preset_normal_env
from nmdc_runtime.resources.core import get_runtime_api_site_client


client = get_runtime_api_site_client(run_config_frozen__preset_normal_env)

In [ ]:
# import os

# from nmdc_runtime.resources.core import RuntimeApiSiteClient

# client = RuntimeApiSiteClient(
#     base_url="https://drs.microbiomedata.org",
#     site_id=os.getenv("API_SITE_ID"),
#     client_id=os.getenv("API_SITE_CLIENT_ID"),
#     client_secret=os.getenv("API_SITE_CLIENT_SECRET"),
# )

# Register objects with existing URLs
The mongoexports at https://portal.nersc.gov/project/m3408/meta/mongoexports/2021-07/

In [ ]:
from pprint import pprint

nmdc_db_collection_names = set(nmdc_jsonschema["definitions"]["Database"]["properties"])
nmdc_db_collection_names -= {
    # not actually collections
    "activity_set",
    "nmdc_schema_version",
    "date_created",
    "etl_software_version",
}

pprint(nmdc_db_collection_names)

```bash
scp -r dtn01.nersc.gov:/project/projectdirs/m3408/www/meta/mongoexports/2021-07/ \
    /Users/dwinston/nmdc_files/mongoexports/
```

In [ ]:
from glob import glob

from nmdc_runtime.util import drs_metadata_for
import requests
from tqdm.notebook import tqdm

paths = glob(os.path.expanduser("~/nmdc_files/mongoexports/2021-07/*"))
url_base = "https://portal.nersc.gov/project/m3408/meta/mongoexports/2021-07"
objects = []
for path in tqdm(paths):
    meta = drs_metadata_for(path)
    url = f'{url_base}/{meta["name"]}'
    
    rv = requests.head(
        url, allow_redirects=True, timeout=5, headers={"Accept-Encoding": "gzip;q=0"}
    )
    if not rv.status_code == 200:
        raise Exception(f"url {url} not OK")

    try:
        size_bytes = int(rv.headers['Content-Length'])
    except KeyError:
        raise Exception(f"no content-length response for {url}")
        
    if size_bytes != meta["size"]:
        raise Exception(f'size of local file {path} ({meta["size"]}) does not match size at {url} ({size_bytes})')
    
    objects.append(DrsObjectIn(
        access_methods=[{"access_url": {"url": url}}],
        **meta
    ))

In [ ]:
rvs = []
for doc in tqdm([json.loads(o.json(exclude_unset=True)) for o in objects]):
    rvs.append(client.create_object(doc))

# Register a bundle

In [ ]:
# checksum computed over sorted concatenation of the corresponding checksums of its top-level contents
# size is cumulative size of contents
# created_time must be >= max of contents created_time values

from glob import glob
import os
from pathlib import Path

from toolz import dissoc

paths = glob(os.path.expanduser("~/nmdc_files/mongoexports/2021-07/*"))
names = [Path(p).name for p in paths]
bundle_content_objects = [
    dissoc(db.objects.find_one(
        filter={"name": n},
        projection=["id", "name", "size", "checksums", "created_time"],
        sort=[("created_time", -1)]
    ), "_id")
    for n in names
]

In [ ]:
checksums = [
    c["checksum"]
    for bco in bundle_content_objects
    for c in bco["checksums"]
    if c["type"] == "sha-256"
]
if len(checksums) != len(bundle_content_objects):
    raise Exception("Contents of bundle must have sha-256 checksums to compute bundle checksum")

In [ ]:
from nmdc_runtime.api.core.util import hash_from_str

checksum = hash_from_str("".join(sorted(checksums)), algo="sha256")

In [ ]:
size = sum(o["size"] for o in bundle_content_objects)

In [ ]:
created_time = max(o["created_time"] for o in bundle_content_objects)

In [ ]:
checksum, size, created_time

In [ ]:
from nmdc_runtime.api.core.util import pick
from nmdc_runtime.api.models.object import DrsObjectIn

drs_bundle_in = DrsObjectIn(
    name="2021-07-nmdc-dbexport",
    description="bzip2-encoded JSON Lines mongoexports of the NMDC database",
    checksums=[{"type": "sha-256", "checksum": checksum}],
    contents=[pick(["id", "name"], o) for o in bundle_content_objects],
    created_time=created_time,
    size=size,
)

In [ ]:
import json
from pprint import pprint

pprint(json.loads(drs_bundle_in.json(exclude_unset=True)))

In [ ]:
rv = client.create_object(json.loads(drs_bundle_in.json(exclude_unset=True)))

In [ ]:
rv.json()

# Register objects by S3 upload and access IDs

In [ ]:
from pathlib import Path

from dagster import build_solid_context
from nmdc_runtime.solids.core import local_file_to_api_object as lftao

context = build_solid_context(
    resources={"mongo": mongo, "runtime_api_site_client": client}
)

storage_path = "../src/data/2021-07-02-study-changes.csv"


def local_file_to_api_object(file_info):
    return lftao(context, file_info)

obj = local_file_to_api_object({"storage_path": storage_path, "mime_type": 'text/csv'})

doc = db.objects.find_one({"id": obj["id"]})
assert doc["name"] == Path(storage_path).name

In [ ]:
obj["id"]

In [ ]:
!echo '{"hello": "donny"}' > test.json

In [ ]:
rv = client.put_object_in_site({"mime_type": "application/json", "name": "test.json"})

In [ ]:
op = rv.json()

In [ ]:
import put_object

rv = put_object("test.json", op["metadata"]["url"])
rv

In [ ]:
from nmdc_runtime.util import drs_object_in_for

result = drs_object_in_for("test.json", op)

In [ ]:
import json
from pprint import pprint

op_patch = {"done": True, "result": result}

In [ ]:
rv = client.update_operation(op["id"], op_patch)

In [ ]:
pprint(rv.json())

# GSP schema  / Cordra stuff

Need "id" in payload, e.g.:
```
"results": [
    {
      "id": "test/activity",
      "type": "Schema",
      "content": {
        "name": "Activity",
        "schema": collschemas["activity_set"]
      }
    }
  ]
```

In [ ]:
from time import time
import os

tic = time()

from dotenv import load_dotenv
load_dotenv(os.path.expanduser("~/.nmdc_mongo.env"))

In [ ]:
os.environ["NMDC_JSON_SCHEMA_FILE"] = "/Users/dwinston/Desktop/nmdc.schema.gsp.json"

In [ ]:
import json
import re
from toolz import assoc_in, dissoc
from zipfile import ZipFile

from mongospawn.schema import collschemas_for

from nmdc_mongo import (
    add_to_db,
    correct_metaP_doc,
    dbschema,
    fetch_and_validate_json,
    fetch_conform_and_persist_from_manifest,
    fetch_json,
    get_db,
    reset_database,
    snake_case_set_name
)

In [ ]:
###########################
# Adjustments for GSP below
###########################

defined_object_names = set(dbschema["definitions"])

set_for_object_name = {
    spec["items"]["$ref"].split("#/definitions/")[-1]: set_name
    for set_name, spec in dbschema["properties"].items()
}

existing_set_names = set(dbschema["properties"])

for object_without_set in (defined_object_names - set(set_for_object_name.keys())):
    proposed_set_name = snake_case_set_name(object_without_set)
    if proposed_set_name not in existing_set_names:
        dbschema["properties"][proposed_set_name] = {
            "description": (f"This property links a database object to the set of"
                            f" {object_without_set} objects within it."),
            "items": {"$ref": f"#/definitions/{object_without_set}"},
            "type": "array",
        }
        
dbschema = assoc_in(dbschema, ["definitions", "ControlledTermValue", "properties", "term", "type"], "string")
del dbschema["definitions"]["ControlledTermValue"]["properties"]["term"]["$ref"]

# 'k' not capitalized upstream perhaps. should conform!
#dbschema = assoc_in(dbschema, ["definitions", "MetagenomeAssembly", "properties", "scaf_l_gt50k", "type"], "number")

In [ ]:
collschemas = collschemas_for(dbschema)

# Reconstruct
set_for_object_name = {
    spec["items"]["$ref"].split("#/definitions/")[-1]: set_name
    for set_name, spec in dbschema["properties"].items()
}

In [ ]:
sorted(collschemas.keys())

In [ ]:
collschemas["biosample_set"]

In [ ]:
import requests

In [ ]:
rv = requests.post("http://localhost:8080/auth/token",
              {"grant_type": "password", "username": "admin", "password": "nmdcrulez"})

In [ ]:
rv.json()

In [ ]:
auth_header = {"Authorization": f'Bearer {rv.json()["access_token"]}'}

In [ ]:
auth_header

In [ ]:
rv = requests.post("http://localhost:8080/uploadObjects", json={
    "results": [
        {
          "type": "Schema",
          "content": {
            "name": "Biosample",
            "schema": collschemas["biosample_set"]
          }
        }
      ]
}, headers=auth_header)

In [ ]:
rv

In [ ]:
import json
with open('/Users/dwinston/Desktop/cordra-upload.json','w') as f:
    json.dump({
        "results": [
            {
              "id": "test/activity",
              "type": "Schema",
              "content": {
                "name": "Activity",
                "schema": collschemas["activity_set"]
              }
            }
          ]
    }, f, indent=2)

In [ ]:
import json
with open('/Users/dwinston/Desktop/cordra-upload.json','w') as f:
    json.dump({
        "results": [
            {
              "id": "test/study",
              "type": "Schema",
              "content": {
                "name": "Study",
                "schema": collschemas["study_set"]
              }
            }
          ]
    }, f, indent=2)

In [ ]:
rv = requests.get("http://localhost:8080/search?query=type:%22Schema%22", headers=auth_header)

In [ ]:
from pprint import pprint

template = rv.json()
del template["pageNum"]
del template["pageSize"]
del template["size"]
del template["results"][0]
del template["results"][0]
template["results"][0]["id"] = "test/abcd1234"
template["results"][0]["content"]["name"] = "Document2"
template["results"][0]["content"]["schema"]["title"] = "Document2"
del template["results"][0]["content"]["identifier"]

pprint(template)

In [ ]:
import json
with open('/Users/dwinston/Desktop/cordra-upload.json','w') as f:
    json.dump(template, f, indent=2)

In [ ]:
!cat /Users/dwinston/Desktop/cordra-upload.json

In [ ]:
rv = requests.post("http://localhost:8080/uploadObjects", json=template, headers=auth_header)
rv